In [77]:
import numpy as np
import pandas as pd
import graph_tool.all as gt

In [25]:
import os
import json

In [3]:
path = '../../data/us/raw'

In [84]:
for session in ['113', '114', '115']:
    # get edges
    cosponsorships = pd.concat([pd.read_csv(os.path.join(path,session,chamber,'cosponsorships.csv'),header=None)
                                        for chamber in os.listdir(os.path.join(path, session))])    
    
    # make graph
    g = gt.Graph(directed=False)
    g.vp['id'] = g.add_edge_list(cosponsorships.values, hashed=True)
    gt.remove_parallel_edges(g)
    gt.remove_self_loops(g)
    
    # add party property map
    parties = pd.read_csv(f'{path}/legislators/terms.csv')
    vertex_df = pd.DataFrame({'id': list(g.vp['id'])})
    mapped_df = vertex_df.merge(parties,left_on='id', right_on='bioguide_id', how='left')
    g.vp['party'] = g.new_vp('string', mapped_df['party'].values)
    
    g.save(f'../../data/us/graphs/us.{session}.graphml')